<a href="https://colab.research.google.com/github/quoctrung2005/TTNT/blob/main/caro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
TIC TAC TOE / CARO (ALPHA-BETA + DEPTH + LEVEL)
--------------------------------------------
- Nhập 1 số duy nhất thay cho (row, col)
- Chế độ 1 người (AI) hoặc 2 người
- Chọn kích thước bàn cờ n x n
- Chọn ai đi trước
- AI có độ khó: Easy / Medium / Hard
"""

import copy
import math

# ==============================
#  HẰNG SỐ
# ==============================
X = "X"
O = "O"
EMPTY = None

human = None
ai = None

LEVEL_DEPTH = {
    "1": 1,   # Easy
    "2": 3,   # Medium
    "3": 5    # Hard
}


# ==============================
#  KHỞI TẠO BÀN CỜ
# ==============================
def initial_state(n):
    return [[EMPTY for _ in range(n)] for _ in range(n)]


# ==============================
#  LƯỢT CHƠI
# ==============================
def player(board):
    moves = sum(cell is not EMPTY for row in board for cell in row)
    return X if moves % 2 == 0 else O


# ==============================
#  NƯỚC ĐI HỢP LỆ
# ==============================
def actions(board, n):
    return [(i, j) for i in range(n) for j in range(n) if board[i][j] is EMPTY]


# ==============================
#  TẠO TRẠNG THÁI MỚI
# ==============================
def result(board, action):
    i, j = action
    new_board = copy.deepcopy(board)
    new_board[i][j] = player(board)
    return new_board


# ==============================
#  KIỂM TRA THẮNG
# ==============================
def winner(board, n):
    lines = []

    for i in range(n):
        lines.append(board[i])
        lines.append([board[j][i] for j in range(n)])

    lines.append([board[i][i] for i in range(n)])
    lines.append([board[i][n - i - 1] for i in range(n)])

    for line in lines:
        if line[0] is not EMPTY and line.count(line[0]) == n:
            return line[0]
    return None


def terminal(board, n):
    return winner(board, n) is not None or all(
        cell is not EMPTY for row in board for cell in row
    )


# ==============================
#  ĐIỂM CUỐI
# ==============================
def utility(board, n):
    w = winner(board, n)
    if w == X:
        return 1
    if w == O:
        return -1
    return 0


# ==============================
#  HEURISTIC
# ==============================
def heuristic(board, n):
    score = 0
    lines = []

    for i in range(n):
        lines.append(board[i])
        lines.append([board[j][i] for j in range(n)])

    lines.append([board[i][i] for i in range(n)])
    lines.append([board[i][n - i - 1] for i in range(n)])

    for line in lines:
        if O not in line:
            score += line.count(X)
        if X not in line:
            score -= line.count(O)

    return score / (n * n)


# ==============================
#  ALPHA-BETA + DEPTH
# ==============================
def max_value(state, n, alpha, beta, depth):
    if terminal(state, n):
        return utility(state, n)
    if depth == 0:
        return heuristic(state, n)

    v = -math.inf
    for a in actions(state, n):
        v = max(v, min_value(result(state, a), n, alpha, beta, depth - 1))
        alpha = max(alpha, v)
        if alpha >= beta:
            break
    return v


def min_value(state, n, alpha, beta, depth):
    if terminal(state, n):
        return utility(state, n)
    if depth == 0:
        return heuristic(state, n)

    v = math.inf
    for a in actions(state, n):
        v = min(v, max_value(result(state, a), n, alpha, beta, depth - 1))
        beta = min(beta, v)
        if alpha >= beta:
            break
    return v


def minimax(board, n, depth_limit):
    current = player(board)
    best_move = None

    if current == X:
        best = -math.inf
        for a in actions(board, n):
            val = min_value(result(board, a), n, -math.inf, math.inf, depth_limit - 1)
            if val > best:
                best = val
                best_move = a
    else:
        best = math.inf
        for a in actions(board, n):
            val = max_value(result(board, a), n, -math.inf, math.inf, depth_limit - 1)
            if val < best:
                best = val
                best_move = a

    return best_move


# ==============================
#  HIỂN THỊ BÀN CỜ (1 SỐ)
# ==============================
def print_board(board, n):
    print("\nBàn cờ:")
    for i in range(n):
        row = [board[i][j] if board[i][j] else "." for j in range(n)]
        print(" ".join(row))

def print_position_guide(n):
    print("\nQuy ước đánh số ô:")
    num = 1
    for i in range(n):
        row = []
        for j in range(n):
            row.append(str(num))
            num += 1
        print(" ".join(f"{c:>2}" for c in row))



# ==============================
#  MAIN
# ==============================
if __name__ == "__main__":
    print("=== TIC TAC TOE / CARO ===")

    n = int(input("Nhập kích thước bàn cờ (n x n): "))
    mode = input("Chế độ (1 = AI, 2 = 2 người): ")

    depth_limit = 0
    if mode == "1":
        print("Chọn độ khó: 1 = Easy | 2 = Medium | 3 = Hard")
        level = input("Độ khó: ")
        depth_limit = LEVEL_DEPTH.get(level, 3)

        first = input("Ai đi trước? (1 = Người, 2 = AI): ")
        if first == "1":
            human, ai = X, O
        else:
            human, ai = O, X

    board = initial_state(n)
    print_position_guide(n)
    while True:
        print_board(board, n)

        if terminal(board, n):
            w = winner(board, n)
            print("\nKẾT THÚC")
            print("Hòa!" if w is None else f"{w} thắng!")
            break

        current = player(board)
        print(f"\nLượt của {current}")

        if mode == "1" and current == ai:
            print("AI đang suy nghĩ...")
            move = minimax(board, n, depth_limit)
            board = result(board, move)
        else:
            while True:
                try:
                    pos = int(input("Chọn ô (số): ")) - 1
                    i, j = divmod(pos, n)
                    if not (0 <= pos < n * n):
                        raise ValueError
                    if board[i][j] is not EMPTY:
                        raise ValueError
                    board = result(board, (i, j))
                    break
                except:
                    print("Ô không hợp lệ, nhập lại!")

=== TIC TAC TOE / CARO ===
Nhập kích thước bàn cờ (n x n): 3
Chế độ (1 = AI, 2 = 2 người): 2

Quy ước đánh số ô:
 1  2  3
 4  5  6
 7  8  9

Bàn cờ:
. . .
. . .
. . .

Lượt của X
Chọn ô (số): 1

Bàn cờ:
X . .
. . .
. . .

Lượt của O
Chọn ô (số): 2

Bàn cờ:
X O .
. . .
. . .

Lượt của X
Chọn ô (số): 3

Bàn cờ:
X O X
. . .
. . .

Lượt của O
Chọn ô (số): 4

Bàn cờ:
X O X
O . .
. . .

Lượt của X
Chọn ô (số): 5

Bàn cờ:
X O X
O X .
. . .

Lượt của O
Chọn ô (số): 6

Bàn cờ:
X O X
O X O
. . .

Lượt của X
Chọn ô (số): 7

Bàn cờ:
X O X
O X O
X . .

KẾT THÚC
X thắng!
